In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from textblob import TextBlob
from sentence_transformers import SentenceTransformer

In [19]:
# Initialize Spark session
spark = SparkSession.builder.appName("0000.parquet").getOrCreate()

In [20]:
# Read Parquet file into DataFrame
df = spark.read.parquet("0000.parquet")

In [21]:
# Show DataFrame
df.show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|I rented I AM CUR...|    0|
|"I Am Curious: Ye...|    0|
|If only to avoid ...|    0|
|This film was pro...|    0|
|Oh, brother...aft...|    0|
|I would put this ...|    0|
|Whoever wrote the...|    0|
|When I first saw ...|    0|
|Who are these "Th...|    0|
|This is said to b...|    0|
|It was great to s...|    0|
|I can't believe t...|    0|
|Never cast models...|    0|
|Its not the cast....|    0|
|Today I found "Th...|    0|
|This film is just...|    0|
|My interest in Do...|    0|
|I have this film ...|    0|
|I think I will ma...|    0|
|Pros: Nothing<br ...|    0|
+--------------------+-----+
only showing top 20 rows

